# Defenser OLS
- Import Package
- Connect DB & get Forword Player Data
- Scaling
- Summary OLS 
- Remove Feature
- Anova & Remove Feature

### Import Package

In [2]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import MySQLdb

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

### Connect DB & get Forword Player Data

In [3]:
db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "football",
    charset='utf8',
)

def make_query(position):
    """
    
    parameter------------
    position : M, D, F, G
    
    return---------------
    SQL_QUERY String
    
    """
    SQL_QUERY = """
        SELECT 
            age, tall, weight, apps_start, apps_sub, mins, goals, assists, yel, red
            , spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb, blocks
            , owng, keyp_x, fouled, off, disp, unstch, avgp, ps_y, rating
        FROM player
        WHERE position like "%{position}%"
        ;
    """.format(position=position)
    
    return SQL_QUERY

# forword
SQL_QUERY = make_query("D")
defenser_df = pd.read_sql(SQL_QUERY, db)

len(defenser_df)

1480

##### Scaling

In [4]:
X = defenser_df.ix[:,:-1]
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

dfX0 = pd.DataFrame(X_scaled, columns=X.columns)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(defenser_df.ix[:,-1], columns=["rating"])
d_df = pd.concat([dfX, dfy], axis=1)
d_df.head()

,const,age,tall,weight,apps_start,apps_sub,mins,goals,assists,yel,...,blocks,owng,keyp_x,fouled,off,disp,unstch,avgp,ps_y,rating
0,1,6.507281,8.489754,6.359613,2.810040,1.504249,2.870309,1.719469,2.304474,2.094252,...,1.300152,0.000000,1.038832,1.142950,0.000000,1.541010,1.708279,3.207327,7.738772,7.05
1,1,6.507281,8.215891,6.277021,3.091044,0.376062,3.078303,0.859734,2.304474,2.094252,...,0.325038,0.000000,1.817956,1.142950,1.151704,1.797845,1.952319,2.481688,6.942278,7.02
2,1,5.302229,8.626686,7.020352,2.341700,0.376062,2.361525,2.579203,0.768158,2.443294,...,1.950229,2.904089,0.519416,1.904916,0.000000,0.513670,0.732120,3.613686,7.886270,6.93
3,1,6.748291,8.078960,6.194428,3.091044,0.376062,3.099635,0.000000,2.304474,2.094252,...,0.325038,0.000000,1.817956,1.333441,0.000000,1.541010,1.708279,2.271252,7.630606,6.90
4,1,6.507281,8.261535,6.359613,1.405020,3.008499,1.559417,0.000000,0.000000,0.349042,...,0.325038,0.000000,1.038832,0.761966,1.151704,0.770505,1.220200,2.576021,7.581439,6.89


##### Summary OLS

In [5]:
model = sm.OLS(d_df.ix[:, -1], d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     187.3
Date:                Sat, 25 Jun 2016   Prob (F-statistic):               0.00
Time:                        19:05:23   Log-Likelihood:                 679.63
No. Observations:                1480   AIC:                            -1301.
Df Residuals:                    1451   BIC:                            -1148.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.8743      0.052    112.334      0.0

##### Remove Some Feature

In [6]:
# remove features
remove_column_list = [
    "age", "tall", "weight", "apps_start", "apps_sub", "mins", "yel", "off"
]
removed_d_df = d_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed_d_df.ix[:, -1], removed_d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     260.3
Date:                Sat, 25 Jun 2016   Prob (F-statistic):               0.00
Time:                        19:06:44   Log-Likelihood:                 672.11
No. Observations:                1480   AIC:                            -1302.
Df Residuals:                    1459   BIC:                            -1191.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.8568      0.037    159.663      0.0

##### Anova &  Remove Feature

In [7]:
formula_str = """
rating ~ goals + assists + red + spg + ps_x + motm + aw
+ tackles + inter + fouls + offsides + clear + drb + blocks
+ owng + keyp_x + fouled + disp + unstch + avgp + ps_y
"""

model = sm.OLS.from_formula(formula_str, data=removed_d_df)
result = model.fit()
table_anova = sm.stats.anova_lm(result)
table_anova

,df,sum_sq,mean_sq,F,PR(>F)
goals,1.0,23.683249,23.683249,988.913961,3.696272e-166
assists,1.0,11.733409,11.733409,489.938376,7.751786e-94
red,1.0,0.265288,0.265288,11.077324,8.956675e-04
spg,1.0,2.159947,2.159947,90.190400,8.501854e-21
ps_x,1.0,7.417041,7.417041,309.704771,5.135255e-63
motm,1.0,16.657729,16.657729,695.557458,1.143787e-125
aw,1.0,19.626518,19.626518,819.521760,2.049732e-143
tackles,1.0,20.187575,20.187575,842.949177,1.166734e-146
inter,1.0,10.720732,10.720732,447.653178,7.139513e-87
fouls,1.0,0.743583,0.743583,31.048932,2.992062e-08


In [9]:
# remove feature 2
remove_column_list = [
    "red", "offsides", "drb", "blocks", "disp", "ps_y"
]
removed2_d_df = removed_d_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed2_d_df.ix[:, -1], removed2_d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     321.9
Date:                Sat, 25 Jun 2016   Prob (F-statistic):               0.00
Time:                        19:10:53   Log-Likelihood:                 627.02
No. Observations:                1480   AIC:                            -1222.
Df Residuals:                    1464   BIC:                            -1137.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.8544      0.038    156.092      0.0